In [15]:
import argparse 
import tensorflow as tf
from load_graph import load_graph
import dataset_loading
import numpy as np
import functions

# We use our "load_graph" function
graph = load_graph("model/affordance-cvae-27000-2-final.ckpt.bytes")

# load data
vr_data = dataset_loading.read_timeseries_data_only_train('VR_data')

# We can verify that we can access the list of operations in the graph
# for op in graph.get_operations():
#     print(op.name)
    # prefix/Placeholder/inputs_placeholder
    # ...
    # prefix/Accuracy/predictions

# We access the input and output nodes 
input_dims = 27000
cond_dims = 25
X_in = graph.get_tensor_by_name('prefix/X:0')
Cond = graph.get_tensor_by_name('prefix/Cond:0')
dec = graph.get_tensor_by_name('prefix/decoder/Reshape_1:0')
keep_prob = graph.get_tensor_by_name('prefix/keep_prob:0')
batch_size = 64





Loading VR_data/train-series-float32.timeseries
Loading VR_data/train-labels-float32.labels


In [17]:
# We launch a Session
predictions = []
batches = []
errors = []
batches_completed = 0
with tf.Session(graph=graph) as sess:
#     while vr_data.train.num_examples - batches_completed * batch_size > 0:
    batch, batch_labels = vr_data.train.next_timeseries_batch(batch_size=batch_size)
    batch = np.reshape(batch, [-1, input_dims])
    batch_labels = np.reshape(batch_labels, [-1, cond_dims])
    d = sess.run([dec], feed_dict={X_in: batch, Cond: batch_labels, keep_prob: 0.8})
    predictions.append(d[0])
    batches.append(batch)
    e = functions.fastDTWDistance_v1(d[0], batch)
    errors = errors + e
    batches_completed += 1
    print(errors)

sess.close()


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
np_pred = np.array(predictions)
np_batches = np.array(batches)
np_errors = np.array([i[0] for i in errors])
np.save("predictions.npy", np_pred)
np.save("batches.npy", np_batches)
np.save("errors.npy", np_errors)

ValueError: setting an array element with a sequence.